In [155]:
import string
import math
from pystempel import Stemmer
from itertools import groupby

stemmer = Stemmer.default()

file = open("pan-tadeusz.txt", "r")
text = file.read()
file.close()

def extracact_words(text):
    words = text.split()

    words = [word.lower() for word in words]
    words = [word.strip(string.punctuation) for word in words]
    words = [word.strip("—»«…") for word in words]
    words = [word.strip(string.punctuation) for word in words]
    words = [word for word in words if word]
    words = [word for word in words if not word.isnumeric()]

    stop_words_file = open("stop-words.txt", "r")
    stop_words = stop_words_file.read().split()
    stop_words_file.close()

    words = [stemmer(word) for word in words]
    words = [word for word in words if word not in stop_words]

    # drop None values
    words = [word for word in words if word]

    return words

words = extracact_words(text)

print(words)

Loading: 100%|██████████| 2225192/2225192 [00:00<00:00, 3814860.17bytes/s]


['adać', 'mickiewicz', 'tadeusz', 'czyć', 'ostatni', 'zajazd', 'litwa', 'księgać', 'pierwy', 'gospodarstwo', 'powrót', 'panicz', 'spotkać', 'pierwy', 'pokoik', 'drugie', 'stołu', 'ważny', 'sędzi', 'nauka', 'grzeczność', 'podkomorzy', 'uwaga', 'polityczny', 'modami', 'początek', 'spor', 'kusy', 'eć', 'żały', 'wojski', 'ostatni', 'woźny', 'trybunał', 'rzucit', 'oko', 'ówczesny', 'stan', 'polityczny', 'litwa', 'europa', 'liwać', 'ojczyznyć', 'y', 'zdrowie', 'cenić', 'tylkć', 'dowie', 'stąić', 'dziść', 'piękność', 'twą', 'ca', 'ozdobie', 'widzieć', 'opisywać', 'tęsknić', 'panna', 'świę', 'jasny', 'bronić', 'częstochowy', 'ostry', 'świecić', 'bram', 'groda', 'zamkowy', 'nowogródzki', 'ochraniać', 'wierny', 'lud', 'mn', 'dziecko', 'zdrowia', 'powrócić', 'cudo', 'płaczęcy', 'mat', 'twoon', 'opieka', 'ofiarować', 'martwy', 'podnieść', 'powieić', 'eć', 'móc', 'pieszo', 'twy', 'świątynia', 'próg', 'ist', 'wrócony', 'żyto', 'podziękować', 'bóg', 'tk', 'us', 'powrócić', 'cudo', 'ojczyzna', 'łon', 

In [137]:
def count_words(words):
    pairs = [(w, 1) for w in words]
    pairs.sort()

    word = lambda pair: pair[0]
    grouped_pairs = [(w, sum(1 for _ in g)) for w, g in groupby(pairs, key=word)]

    return grouped_pairs

In [87]:
words_count = count_words(words)
words_count.sort(key=lambda pair: pair[1], reverse=True)
words_count = words_count[4:150]
print(words_count)

[('hrabia', 196), ('swój', 180), ('tadeusz', 174), ('mieć', 173), ('głowa', 172), ('wielki', 169), ('chcieć', 149), ('widzieć', 147), ('raz', 145), ('szlachta', 133), ('wiedzieć', 126), ('choć', 121), ('mówić', 116), ('wojski', 112), ('długi', 108), ('czas', 100), ('być', 96), ('dom', 95), ('iść', 93), ('mój', 93), ('pot', 87), ('myślić', 84), ('strona', 84), ('cały', 83), ('gość', 83), ('pan', 83), ('zamek', 83), ('serce', 82), ('ziemia', 82), ('koniec', 81), ('zosie', 78), ('świat', 78), ('drugi', 77), ('słowo', 76), ('telimena', 76), ('krzyknąć', 74), ('pierwszy', 74), ('tył', 74), ('stać', 73), ('bóg', 72), ('klucznik', 71), ('dwór', 69), ('móc', 69), ('patrzyć', 69), ('robak', 69), ('dawny', 68), ('rzecz', 68), ('brat', 67), ('ksiądz', 67), ('zacząć', 67), ('ów', 67), ('biały', 66), ('cóż', 66), ('podkomorzy', 66), ('prawy', 66), ('znać', 64), ('cichy', 63), ('wziąć', 62), ('koń', 61), ('moskal', 61), ('gerwaza', 60), ('noga', 60), ('stół', 60), ('oba', 59), ('litwa', 58), ('nowy'

In [88]:
# save to file
file = open("output.txt", "w")
for word, _ in words_count:
    file.write(f"{word}\n")

file.close()

In [89]:
documents = ["księga-1", "księga-2", "księga-3", "księga-4", "księga-5", "księga-6", "księga-7", "księga-8", "księga-9", "księga-10", "księga-11", "księga-12", "epilog"]

words_in_documents = {}

for document in documents:
    file = open(f"rozdziały/{document}.txt", "r")
    text = file.read()

    words = extracact_words(text)
    words_count = count_words(words)

    words_in_documents[document] = words_count

    file.close()

In [94]:
tf_idf_dict = {}
for document in documents:

    tf_idf_dict[document] = {}

    for word, count in words_in_documents[document]:
        tf = count
        
        counter = 0
        for other_document in documents:
            if other_document == document:
                continue

            if word in dict(words_in_documents[other_document]):
                counter += 1

        idf = math.log(len(documents) / (1 + counter))
        tf_idf = tf * idf
        tf_idf_dict[document][word] = tf_idf

print(tf_idf_dict)

{'księga-1': {'adiutant': 2.5649493574615367, 'adwokat': 2.5649493574615367, 'afekt': 1.466337068793427, 'alkowa': 1.466337068793427, 'alp': 2.5649493574615367, 'alternata': 2.5649493574615367, 'ambona': 1.466337068793427, 'angielski': 0.9555114450274363, 'architekt': 2.5649493574615367, 'architektura': 1.8718021769015913, 'armistycjum': 2.5649493574615367, 'asesor': 2.206348680751904, 'astra': 2.5649493574615367, 'austerlitz': 1.8718021769015913, 'autor': 2.5649493574615367, 'awanpost': 2.5649493574615367, 'ażeby': 0.5247285289349821, 'baczność': 2.5649493574615367, 'bagno': 1.466337068793427, 'bal': 1.8718021769015913, 'baron': 5.1298987149230735, 'barwa': 1.466337068793427, 'bawić': 1.4708991205012694, 'bazyliszek': 2.5649493574615367, 'baśń': 2.5649493574615367, 'beczeć': 1.466337068793427, 'bernardyn': 1.3118213223374553, 'bernardyński': 1.8718021769015913, 'bernatowicz': 2.5649493574615367, 'bezładność': 2.5649493574615367, 'bezłać': 1.8718021769015913, 'białopiotrowicz': 1.87180

In [98]:
for document in documents:
    file = open(f"wordclouds/{document}.txt", "w")
    
    words = tf_idf_dict[document]
    words = [(word, tf_idf) for word, tf_idf in words.items()]
    words.sort(key=lambda pair: pair[1], reverse=True)
    words = words[:150]

    for word, _ in words:
        file.write(f"{word}\n")

    file.close()

In [117]:
def rank_documents(word, tf_idf_dict):
    ranks = []

    for document in documents:
        if word in tf_idf_dict[document]:
            ranks.append((document, tf_idf_dict[document][word]))
        else:
            ranks.append((document, 0))

    ranks.sort(key=lambda pair: pair[1], reverse=True)
    return ranks

In [118]:
ranks = rank_documents("wojski", tf_idf_dict)

print(ranks)

[('księga-4', 4.343406201242322), ('księga-12', 2.6728653546106598), ('księga-5', 2.5058112699474937), ('księga-11', 2.338757185284327), ('księga-1', 2.0046490159579946), ('księga-2', 1.5034867619684962), ('księga-8', 1.1693785926421636), ('księga-3', 0.8352704233158312), ('księga-9', 0.8352704233158312), ('księga-10', 0.33410816932633247), ('księga-6', 0.16705408466316624), ('księga-7', 0), ('epilog', 0)]


In [156]:
file = open("pan-tadeusz.txt", "r")
text = file.read()
file.close()

words = extracact_words(text)

word_pairs = [(words[i], words[i + 1]) for i in range(len(words) - 1)]

word_pairs_count = count_words(word_pairs)

In [157]:
next_words = {}

for (word, next_word), count in word_pairs_count:
    if word not in next_words:
        next_words[word] = []

    if len(next_words[word]) < 5:
        next_words[word].append((next_word, count))
    else:
        # sort by count
        next_words[word].sort(key=lambda pair: pair[1])
        if count > next_words[word][0][1]:
            next_words[word][0] = (next_word, count)


print(next_words)

{'absolucja': [('dobra', 1)], 'abym': [('często', 1), ('rączki', 1), ('służyć', 1)], 'abyś': [('raczyć', 1), ('zostać', 1)], 'achar': [('got', 1)], 'adama': [('al', 1), ('ojec', 1), ('skłócić', 1)], 'adać': [('mickiewicz', 1)], 'adiut': [('sztab', 1)], 'administracja': [('prowadzić', 1)], 'adwokacki': [('wiesz', 1)], 'adwokat': [('rejen', 1)], 'afekt': [('mieć', 1), ('serdeczny', 1), ('stąd', 1)], 'afirmatywa': [('jeżeć', 1)], 'agrest': [('ogrodnik', 1), ('przeskoczyć', 1)], 'agrestowy': [('wiedzący', 1)], 'aha': [('ob', 1)], 'akademia': [('naonczas', 1)], 'akordy': [('fortissiuy', 1), ('muszek', 1), ('rozdąsać', 1), ('syk', 1)], 'akordów': [('wiązać', 1)], 'aksamit': [('trawa', 1), ('zielonej', 1)], 'aksamitny': [('prawy', 1)], 'akt': [('konfederacki', 1), ('mieć', 1), ('spisać', 1), ('urzędowy', 1)], 'akta': [('mieć', 1), ('psuć', 1)], 'aktach': [('jeneralnóść', 1)], 'aktami': [('sławny', 1)], 'aktor': [('scena', 1)], 'al': [('słyszeć', 1), ('tęsknić', 1)], 'alabastrowy': [('stronica

In [159]:
import random

word = "natenczas"

for i in range(100):
    print(word, end=" ")
    if word not in next_words:
        break

    # get random next word
    next_word = next_words[word][random.randint(0, len(next_words[word]) - 1)][0]
    word = next_word


natenczas wojski wciązać gra echo odezwać chór odpowiedzieć doweyko drugi koniec soplica horeśk chociść wojski głowa chcieć konieczny powiedzieć szczero móc dać rozkaz wodz spotkać jakimsić krwawy gerwazeg okazować wielki gniew ludzki gadanie tylkć dziść chcieć konieczny powiedzieć wszystki stan grożyć twarz gniewny oba chór okrzyknąć muzyk mylić znać imia chrzcie dawać czas wojny cesarz idzie jaśn grafie przedć laty piastować królewski dziątać krzykliwy ruchawy czarny kaszkiet trawa bujać kłos złocisty koneway dotąd swój chart jedny drugi czarny oko łza brylant tadeusz telimena pomięszał zdziwić jasny gór jużyć mieć przyjaciela córa nadobny kochanek bonończyk rozkoszny psiny mieć przyjaciela córa hreczeszanka 